In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from AutoReasoner import *

df = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')

In [23]:
def tokenizeDF(df, utterance=None, refine_utt_col_names=True):
    token_dict = {}
    token_id = 0
    cols = [c.lower() for c in df.columns]
    
    for rid in range(df.shape[0]):
        for dtype, c in zip(df.dtypes, df.columns):
            string = df.loc[rid][c] 
            if isinstance(string, float) or isinstance(string, int) or isinstance(string, np.int64) or bool(re.search(r'\d', string)):
                continue
            new_tokens = []
            
            
            # entire string is a single token
            # if string.lower() in token_dict:
            #     new_tokens.append(token_dict[string.lower()])
            # else:
            #     token_dict[string.lower()] = f'TOKEN{token_id}'
            #     new_tokens.append(token_dict[string.lower()])
            #     token_id += 1
            
            # every word is a token
            for token in string.replace('\n', ' ').replace('_', ' ').split(' '):
                if token.lower() in cols:
                    new_tokens.append(token)
                elif token.lower() in token_dict:
                    new_tokens.append(token_dict[token.lower()])
                else:
                    token_dict[token.lower()] = f'TOKEN{token_id}'
                    new_tokens.append(token_dict[token.lower()])
                    token_id += 1
                
            df.at[rid, c] = ' '.join(new_tokens)
    if utterance is not None:
        new_utterance = []
        for u in utterance.strip('?').split(' '):
            if u.lower() in cols and refine_utt_col_names:
                new_utterance.append(df.columns[cols.index(u.lower())])
            elif u.lower() in token_dict:
                new_utterance.append(token_dict[u.lower()] + ' ')
            else:
                new_utterance.append(u)
        utterance = ' '.join(new_utterance) + '?'
    
    return df, utterance, token_dict

def parseTokenizedStr(s, token_dict):
    rev_dict = {v:k for k,v in token_dict.items()}
    new_str = []
    for t in s.split(' '):
        if t in rev_dict:
            new_str.append(rev_dict[t])
        else:
            new_str.append(t)
    return ' '.join(new_str)

i = 3
context_df = pd.read_csv('./dataset/WikiTableQuestions/'+df.iloc[i]['context'], on_bad_lines='skip')
utterance = df.iloc[i]['utterance']
target_val = df.iloc[i]['targetValue']
context_df
# context_df, utterance, target_val
# utterance = 'what is the number of Placing 1 across all events?'

,Series #,Season #,Title,Notes,Original air date
0,1,1,"\The Charity\""""","Alfie, Dee Dee, and Melanie are supposed to be...","October 15, 1994"
1,2,1,"\The Practical Joke War\""""",Alfie and Goo unleash harsh practical jokes on...,"October 22, 1994"
2,3,1,"\The Weekend Aunt Helen Came\""""","The boy's mother, Jennifer, leaves for the wee...","November 1, 1994"
3,4,1,"\Robin Hood Play\""""",Alfie's school is performing the play Robin Ho...,"November 9, 1994"
4,5,1,"\Basketball Tryouts\""""",Alfie tries out for the basketball team and do...,"November 30, 1994"
5,6,1,"\Where's the Snake?\""""","Dee Dee gets a snake, but he doesn't want his ...","December 6, 1994"
6,7,1,"\Dee Dee's Girlfriend\""""",A girl kisses Dee Dee in front of Harry and Do...,"December 15, 1994"
7,8,1,"\Dee Dee's Haircut\""""",Dee Dee wants to get a hair cut by Cool Doctor...,"December 20, 1994"
8,9,1,"\Dee Dee Runs Away\""""",Dee Dee has been waiting to go to a monster tr...,"December 28, 1994"
9,10,1,"'\Donnell's Birthday Party\""""",Donnell is having a birthday party and brags a...,"January 5, 1995"


In [22]:
context_df, utterance, token_dict = tokenizeDFWithColNames(context_df, utterance)
context_df

,Series #,Season #,Title,Notes,Original air date
0,1,1,TITLE0 TITLE1,NOTES0 NOTES1 NOTES2 NOTES3 NOTES4 NOTES5 NOTE...,"October 15, 1994"
1,2,1,TITLE0 TITLE2 TITLE3 TITLE4,NOTES51 NOTES3 NOTES21 NOTES84 NOTES85 TITLE2 ...,"October 22, 1994"
2,3,1,TITLE0 TITLE5 TITLE6 TITLE7 TITLE8,NOTES17 NOTES129 NOTES130 NOTES131 NOTES40 NOT...,"November 1, 1994"
3,4,1,TITLE9 TITLE10 TITLE11,NOTES184 NOTES185 NOTES29 NOTES186 NOTES17 NOT...,"November 9, 1994"
4,5,1,TITLE12 TITLE13,NOTES51 NOTES208 NOTES209 NOTES75 NOTES17 NOTE...,"November 30, 1994"
5,6,1,TITLE14 NOTES17 TITLE15,NOTES1 NOTES1 NOTES147 NOTES13 NOTES233 NOTES4...,"December 6, 1994"
6,7,1,TITLE16 NOTES52 TITLE17,NOTES13 NOTES259 NOTES260 NOTES1 NOTES1 NOTES1...,"December 15, 1994"
7,8,1,TITLE16 NOTES52 TITLE18,NOTES1 NOTES1 NOTES167 NOTES7 NOTES61 NOTES13 ...,"December 20, 1994"
8,9,1,TITLE16 NOTES1 TITLE19 TITLE20,NOTES1 NOTES1 NOTES55 NOTES315 NOTES316 NOTES7...,"December 28, 1994"
9,10,1,TITLE21 TITLE22 TITLE23,NOTES218 NOTES29 NOTES335 NOTES13 TITLE22 NOTE...,"January 5, 1995"


In [55]:
prompter = CodexAnswer(
            df.iloc[i]['id'], 
            df.iloc[i]['utterance'], 
            df.iloc[i]['context'], 
            df.iloc[i]['targetValue'], 
            base_path='./dataset/WikiTableQuestions/',
            demo_file=None
        )

prompter.model = "davinci-codex-002-msft"
prompter.model = "text-davinci-003"

prompter.source_table_df = context_df
prompter.utterance = utterance

prompter._gen_gpt_prompt(demo_num=0)
prompter._get_gpt_prediction()

In [56]:
# prompter.prompt
print(prompter.predicted_result)
parseTokenizedStr(prompter.predicted_result, token_dict), target_val

None


AttributeError: 'NoneType' object has no attribute 'split'

In [19]:
print(prompter.prompt)

A database table is shown as follows:
[HEAD]: Series #|Season #|Title|Notes|Original air date
[ROW] 1: 1|1|\The Charity\""|Alfie, Dee Dee, and Melanie are supposed to be helping their parents at a carnival by working the dunking booth. When Goo arrives and announces their favorite basketball player, Kendall Gill, is at the Comic Book Store signing autographs, the boys decide to ditch the carnival. This leaves Melanie and Jennifer to work the booth and both end up soaked. But the Comic Book Store is packed and much to Alfie and Dee Dee's surprise their father has to interview Kendall Gill. Goo comes up with a plan to get Alfie and Dee Dee, Gill's signature before getting them back at the local carnival, but are caught by Roger. All ends well for everyone except Alfie and Goo, who must endure being soaked at the dunking booth.|October 15, 1994
[ROW] 2: 2|1|\The Practical Joke War\""|Alfie and Goo unleash harsh practical jokes on Dee Dee and his friends. Dee Dee, Harry and Donnel retaliat

In [ ]:
prompt = """
A database table is shown as follows:
[HEAD]: Date|Competition|Location|Country|Event|Placing|Rider|Nationality
[ROW] 1: 31 October 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN3|1|TOKEN4 TOKEN5|TOKEN6
[ROW] 2: 31 October 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN7|2|TOKEN8 TOKEN9|TOKEN6
[ROW] 3: 1 November 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN3|1|TOKEN8 TOKEN9|TOKEN6
[ROW] 4: 1 November 2008|2008–09 World Cup|LOCATION0|COUNTRY0|500 m time trial|1|TOKEN4 TOKEN5|TOKEN6
[ROW] 5: 2 November 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN10 TOKEN3|1|TOKEN11 TOKEN12|TOKEN6
[ROW] 6: 2 November 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN10 TOKEN3|1|TOKEN8 TOKEN9|TOKEN6
[ROW] 7: 2 November 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN10 TOKEN3|1|TOKEN13 TOKEN14|TOKEN6
[ROW] 8: 2 November 2008|2008–09 World Cup|LOCATION0|COUNTRY0|TOKEN7|1|TOKEN4 TOKEN5|TOKEN6
[ROW] 9: 2 November 2008|5th International Keirin Event|LOCATION0|COUNTRY0|TOKEN15 TOKEN7|2|TOKEN11 TOKEN12|TOKEN6
[ROW] 10: 13 February 2009|2008–09 World Cup|LOCATION1|TOKEN17|COUNTRY1|1|TOKEN18 TOKEN19|TOKEN6
[ROW] 11: 13 February 2009|2008–09 World Cup|LOCATION1|TOKEN17|COUNTRY1|1|TOKEN8 TOKEN9|TOKEN6
[ROW] 12: 13 February 2009|2008–09 World Cup|LOCATION1|TOKEN17|COUNTRY1|TOKEN13 TOKEN14|TOKEN6
[ROW] 13: 13 February 2009|2008–09 World Cup|LOCATION1|TOKEN17|TOKEN3|1|TOKEN4 TOKEN5|TOKEN6
[ROW] 14: 30 October 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|TOKEN7|1|TOKEN18 TOKEN19|TOKEN6
[ROW] 15: 30 October 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|TOKEN3|1|TOKEN4 TOKEN5|TOKEN6
[ROW] 16: 30 October 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|TOKEN3|1|TOKEN18 TOKEN19|TOKEN6
[ROW] 17: 30 October 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|500 m time trial|2|TOKEN4 TOKEN5|TOKEN6
[ROW] 18: 1 November 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|TOKEN10 TOKEN3|1|TOKEN11 TOKEN12|TOKEN6
[ROW] 19: 1 November 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|TOKEN10 TOKEN3|1|TOKEN18 TOKEN19|TOKEN6
[ROW] 20: 1 November 2009|2009–10 World Cup|LOCATION0|TOKEN1 TOKEN2|TOKEN10 TOKEN3|1|TOKEN13 TOKEN14|TOKEN6

Answer the following question based on the data above: "what is the number of 1st place finishes across all events?". The answer is: ```
"""

In [3]:
# model = 'text-davinci-003'
model = 'davinci-codex-002-msft'
ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

# CodexAnswerTokenizeDF

In [17]:
def parallel_CodexAnswerTokenizeDF_dynamic_few_shot_func(i):
    try:
        prompter = CodexAnswerTokenizeDF(test_df.iloc[i]['id'], 
                             test_df.iloc[i]['utterance'], 
                             test_df.iloc[i]['context'], 
                             test_df.iloc[i]['targetValue'], 
                             base_path='./dataset/WikiTableQuestions/')
        prompter.model = model
        prompter._gen_NN_demo(
            training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        prompter._gen_gpt_prompt()
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        return prompter._log_dict()
    except Exception as e:
        return {
            'id': test_df.iloc[i]['id'],
            'uncaught_err': str(e)
        } 
    

training_data_autoreasoning = json.load(open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'r'))
all_demo_embeddings = None
print("Loading embeddings")
for train_q in tqdm(training_data_autoreasoning):
    if all_demo_embeddings is None:
        all_demo_embeddings = get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1])
    else:
        all_demo_embeddings = np.append(all_demo_embeddings, get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1]), axis=1)

Loading embeddings


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14149/14149 [00:12<00:00, 1145.23it/s]


In [19]:
test_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='skip')
maxLimit = float('inf')
# maxLimit = 100
n_threads = 20
from joblib import Parallel, delayed
res = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_CodexAnswerTokenizeDF_dynamic_few_shot_func)(i) for i in tqdm(range(min(maxLimit, test_df.shape[0]))))
# json.dump(res, open(f'./dataset/WikiTableQuestions/results/CodexAnswerTokenizeDF_{model}_zeroShot_results_pristine-unseen-tables.json', 'w'))
json.dump(res, open(f'./dataset/WikiTableQuestions/results/CodexAnswerTokenizeDF_{model}_top3cos_results_pristine-unseen-tables.json', 'w'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [08:46<00:00,  8.26it/s]


# CodexAnswer

In [6]:
import time

# model = 'text-davinci-002'
# model = 'text-davinci-003'
model = 'davinci-codex-002-msft'

all_table_formats = {
    'seperators': [',', '|', '\t'],
    'col_data_splits': ['-', ''],
    'col_prefixs' : ['', '[HEAD]'],
    'row_prefixs' : ['', '[ROW]'],
    'key_val_pairs': [True, False]
}

ft = fasttext.load_model('/mnt/idm_automapping/cc.en.300.bin')

def parallel_gpt_answer_dynamic_few_shot_func(i):
    try:
        prompter = GptAnswer(test_df.iloc[i]['id'], 
                             test_df.iloc[i]['utterance'], 
                             test_df.iloc[i]['context'], 
                             test_df.iloc[i]['targetValue'], 
                             base_path='./dataset/WikiTableQuestions/', 
                            table_format = table_format)
        prompter.model = model
        prompter._gen_NN_demo(training_data_autoreasoning, all_demo_embeddings, ft, demo_num=3)
        prompter._gen_gpt_prompt()
        prompter._get_gpt_prediction()
        if prompter.gpt_error is None:
            prompter._evaluate_result(verbose=False)
        log = prompter._log_dict()
    except Exception as e:
        log = {
            'id': test_df.iloc[i]['id'],
            'uncaught_err': str(e)
        }
    if model == 'text-davinci-003':
        time.sleep(5)
    return log

training_data_autoreasoning = json.load(open(f'./dataset/WikiTableQuestions/reasonings/GptReasoner_training.json', 'r'))
all_demo_embeddings = None
print("Loading embeddings")
for train_q in tqdm(training_data_autoreasoning):
    if all_demo_embeddings is None:
        all_demo_embeddings = get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1])
    else:
        all_demo_embeddings = np.append(all_demo_embeddings, get_utterance_embedding(train_q['utterance'], ft).reshape([-1, 1]), axis=1)

test_df = pd.read_csv(f'./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t', on_bad_lines='skip')
maxLimit = float('inf')
# maxLimit = 20 
n_threads = 20
from joblib import Parallel, delayed

for sep in all_table_formats['seperators']:
    for col_data_split in all_table_formats['col_data_splits']:
        for col_prefix in all_table_formats['col_prefixs']:
            for row_prefix in all_table_formats['row_prefixs']:
                # for key_val_pairs in [True, False]:
                for key_val_pairs in [True]:
                    table_format = {
                        'seperator': sep,
                        'col_data_split': col_data_split,
                        'col_prefix' : col_prefix,
                        'row_prefix' : row_prefix,
                        'key_val_pairs': key_val_pairs
                    }
                    arr = [sep.replace('\t', 't'), col_data_split, col_prefix, row_prefix, key_val_pairs]
                    res = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_gpt_answer_dynamic_few_shot_func)(i) for i in tqdm(range(min(maxLimit, test_df.shape[0]))))
                    dump_json_file_name = f'./dataset/WikiTableQuestions/results/GptAnswer_{model}_fewShot_top3cos_sep={arr}_results_pristine-unseen-tables.json'
                    print("Saving to: ", dump_json_file_name)
                    json.dump(res, open(dump_json_file_name, 'w'))

Loading embeddings


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [06:56<00:00, 10.42it/s]


Saving to:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '', True]_results_pristine-unseen-tables.json


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [06:19<00:00, 11.43it/s]


Saving to:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '[ROW]', True]_results_pristine-unseen-tables.json


...

Saving to:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '', '[ROW]', True]_results_pristine-unseen-tables.json


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [06:32<00:00, 11.07it/s]


Saving to:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '', True]_results_pristine-unseen-tables.json


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4344/4344 [07:12<00:00, 10.04it/s]


Saving to:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '[ROW]', True]_results_pristine-unseen-tables.json


In [16]:
import subprocess

from io import StringIO
import sys
mystdout = StringIO()

for sep in all_table_formats['seperators']:
    for col_data_split in all_table_formats['col_data_splits']:
        for col_prefix in all_table_formats['col_prefixs']:
            for row_prefix in all_table_formats['row_prefixs']:
                for key_val_pairs in [True, False]:
                # for key_val_pairs in [True]:
                    table_format = {
                        'seperator': sep,
                        'col_data_split': col_data_split,
                        'col_prefix' : col_prefix,
                        'row_prefix' : row_prefix, 
                        'key_val_pairs': key_val_pairs
                    }
                    if key_val_pairs:
                        arr = [sep.replace('\t', 't'), col_data_split, col_prefix, row_prefix, key_val_pairs]
                    else:
                        arr = [sep.replace('\t', 't'), col_data_split, col_prefix, row_prefix]
                        
                    dump_json_file_name = f'./dataset/WikiTableQuestions/results/GptAnswer_{model}_fewShot_top3cos_sep={arr}_results_pristine-unseen-tables.json'
                    print("Read from: ", dump_json_file_name)
                    out = subprocess.check_output(
                        ['python2', './dataset/WikiTableQuestions/evaluator.py', 
                         '-t', './dataset/WikiTableQuestions/tagged/data/',  
                         f'{dump_json_file_name}']
                    ).decode('utf-8')
                    

Read from:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '', True]_results_pristine-unseen-tables.json


Reading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '', True]_results_pristine-unseen-tables.json
Examples: 4344
Correct: 2047
Accuracy: 0.4712
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged


Read from:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '']_results_pristine-unseen-tables.json


Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '']_results_pristine-unseen-tables.json
Examples: 4344
Correct: 2211
Accuracy: 0.509
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged


Read from:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=[',', '-', '', '[ROW]', True]_results_pristine-unseen-tables.json


...

Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '']_results_pristine-unseen-tables.json
Examples: 4344
Correct: 2231
Accuracy: 0.5136
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged


Read from:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '[ROW]', True]_results_pristine-unseen-tables.json


Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '[ROW]', True]_results_pristine-unseen-tables.json
Examples: 4344
Correct: 2001
Accuracy: 0.4606
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/training.tagged


Read from:  ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '[ROW]']_results_pristine-unseen-tables.json


Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-seen-tables.tagged
Reading dataset from ./dataset/WikiTableQuestions/tagged/data/pristine-unseen-tables.tagged
Read 22033 examples
Reading predictions from ./dataset/WikiTableQuestions/results/GptAnswer_davinci-codex-002-msft_fewShot_top3cos_sep=['t', '', '[HEAD]', '[ROW]']_results_pristine-unseen-tables.json
Examples: 4344
Correct: 2246
Accuracy: 0.517


In [11]:
out

''

In [4]:
prompt = """
The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT name, career_win_loss FROM DF WHERE name="Australian Open" or name="Indian Wells";```.

The database table DF is shown as follows:
[HEAD]: career_win_loss
---
[ROW] 1: Australian Open|22–18
[ROW] 2: Indian Wells|16-13

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

Answer: ```Australian Open```.

The database table DF is shown as follows:
[HEAD]: by_race|white|black|aian*|asian|nhpi*
---
[ROW] 1: 2000 (total population)|75.43%|4.46%|19.06%|5.24%|0.88%
[ROW] 2: 2000 (Hispanic only)|3.42%|0.33%|0.45%|0.16%|0.06%
[ROW] 3: 2005 (total population)|74.71%|4.72%|18.77%|5.90%|0.88%
...
[ROW] 6: Growth 2000–05 (non-Hispanic only)|3.49%|11.30%|4.02%|18.96%|5.86%
[ROW] 7: Growth 2000–05 (Hispanic only)|33.56%|21.02%|14.52%|27.89%|-1.95%

Answer the following question based on the data above: "which hispanic population had the greatest growth from 2000 to 2005?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT * FROM DF WHERE by_race LIKE "Growth 2000–05 (Hispanic only)%";```.

The database table DF is shown as follows:
[HEAD]: by_race|white|black|aian*|asian|nhpi*
---
[ROW] 1: Growth 2000–05 (Hispanic only)|33.56%|21.02%|14.52%|27.89%|-1.95%

Answer the following question based on the data above: "which hispanic population had the greatest growth from 2000 to 2005?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

Answer: ```white```.

The database table DF is shown as follows:
[HEAD]: rank|mountain_peak|mountain_range|elevation|prominence|isolation|location
---
[ROW] 1: 1|Mount Whitney|Sierra Nevada|14,505 ft 4421 m|10,080 ft 3072 m|1,646 mi 2,649 km|36°34′43″N 118°17′31″W﻿ / ﻿36.5786°N 118.2920°W
[ROW] 2: 2|Mount Williamson|Sierra Nevada|14,379 ft 4383 m|1,677 ft 511 m|5.4 mi 8.7 km|36°39′21″N 118°18′40″W﻿ / ﻿36.6559°N 118.3111°W
[ROW] 3: 3|White Mountain Peak|White Mountains|14,252 ft 4344 m|7,196 ft 2193 m|67 mi 109 km|37°38′03″N 118°15′21″W﻿ / ﻿37.6341°N 118.2557°W
...
[ROW] 15: 15|Red Slate Mountain|Sierra Nevada|13,162 ft 4012 m|1,736 ft 529 m|8 mi 13 km|37°30′27″N 118°52′09″W﻿ / ﻿37.5075°N 118.8693°W
[ROW] 16: 16|Mount Ritter|Sierra Nevada|13,149 ft 4008 m|3,990 ft 1216 m|22 mi 35 km|37°41′21″N 119°11′59″W﻿ / ﻿37.6891°N 119.1996°W

Answer the following question based on the data above: "which mountain peak has a prominence more than 10,000 ft?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT mountain_peak, prominence FROM DF;```.

The database table DF is shown as follows:
[HEAD]: mountain_peak|prominence
---
[ROW] 1: Mount Whitney|10,080 ft 3072 m
[ROW] 2: Mount Williamson|1,677 ft 511 m
[ROW] 3: White Mountain Peak|7,196 ft 2193 m
...
[ROW] 15: Red Slate Mountain|1,736 ft 529 m
[ROW] 16: Mount Ritter|3,990 ft 1216 m

Answer the following question based on the data above: "which mountain peak has a prominence more than 10,000 ft?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

Python: ```
def get_ft(s):
    import re
    return float(re.search("(.*?) ft", s).group(1))
DF['prominence_ft'] = DF.apply(lambda x: get_ft(x['prominence']), axis=1)
```.

The database table DF is shown as follows:
[HEAD]: mountain_peak|prominence|prominence_ft
---
[ROW] 1: Mount Whitney|10,080 ft 3072 m|10080.0
[ROW] 2: Mount Williamson|1,677 ft 511 m|1677.0
[ROW] 3: White Mountain Peak|7,196 ft 2193 m|7196.0
...
[ROW] 15: Red Slate Mountain|1,736 ft 529 m|1736.0
[ROW] 16: Mount Ritter|3,990 ft 1216 m|3990.0

Answer the following question based on the data above: "which mountain peak has a prominence more than 10,000 ft?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT mountain_peak FROM DF WHERE prominence_ft>10000;```.

The database table DF is shown as follows:
[HEAD]: mountain_peak
---
[ROW] 1: Mount Whitney

Answer the following question based on the data above: "which mountain peak has a prominence more than 10,000 ft?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

Answer: ```Mount Whitney```.

The database table DF is shown as follows:
[HEAD]: season|division|tms|pos|fa_cup|afc_cl
---
[ROW] 1: 2003|1|12|11|Quarter final|-
[ROW] 2: 2004|1|13|10|Round of 32|-
[ROW] 3: 2005|1|13|8|Quarter final|-
...
[ROW] 9: 2011|1|16|12|Round of 32|-
[ROW] 10: 2012|1|16|10|Round of 16|-

Answer the following question based on the data above: "how far did they make it in the fa cup after 2009?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT fa_cup FROM DF WHERE season>2009;```.

The database table DF is shown as follows:
[HEAD]: fa_cup
---
[ROW] 1: Round of 32
[ROW] 2: Round of 32
[ROW] 3: Round of 16

Answer the following question based on the data above: "how far did they make it in the fa cup after 2009?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

Answer: ```Round of 16```.

The database table DF is shown as follows:
[HEAD]: unnamed_0|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|c_2011|c_2012
---
[ROW] 1: Hydro power|1,150|1,161|1,171|1,247|1,281|1,293|1,316|1,326|1,357|1,379|1,382|1,401|1,584
[ROW] 2: Thermal|685|835|1,056|1,233|1,215|1,155|1,155|1,155|1,285|1,290|1,390|1,690|1,638
[ROW] 3: Other renewables|3|3|3|3|3|3|3|3|3|15|45|50|90
...
[ROW] 7: Other renewables|3|3|4|3|3|5|4|4|8|27|86|121|169
[ROW] 8: Total production|6,685|6,520|6,167|7,611|8,043|8,770|9,389|9,815|9,901|9,883|10,715|11,528|11,800

Answer the following question based on the data above: "did the hydro power increase or decrease from 2010 to 2012?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT c_2010, c_2012 FROM DF WHERE unnamed_0="Hydro power";```.

The database table DF is shown as follows:
[HEAD]: c_2010|c_2012
---
[ROW] 1: 1,382|1,584

Answer the following question based on the data above: "did the hydro power increase or decrease from 2010 to 2012?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

Answer: ```increase```.

The database table DF is shown as follows:
[HEAD]: season|conference|head_coach|total_wins|total_losses|total_ties|conference_wins|conference_losses|conference_ties|conference_standing|postseason_result
---
[ROW] 1: 1905|Independent|Sidney Smith|2|3|1|—|—|—|—|—
[ROW] 2: 1906|Independent|Ralph Foster|3|0|0|—|—|—|—|—
[ROW] 3: 1907|Independent|Ralph Foster|1|5|1|—|—|—|—|—
[ROW] 4: 1908|Southern Intercollegiate|Ralph Foster|4|1|1|—|—|—|—|—
[ROW] 5: 1909|Southern Intercollegiate|Sam Costen|4|3|2|0|1|1|—|—
[ROW] 6: 1910|Southern Intercollegiate|Sam Costen|3|4|0|1|3|0|—|—
[ROW] 7: 1911|Southern Intercollegiate|L. S. LeTellier|5|2|2|1|2|0|—|—
[ROW] 8: 1912|Southern Intercollegiate|L. S. LeTellier|3|4|0|0|3|0|—|—
[ROW] 9: 1913|Southern Intercollegiate|George C. Rogers|3|4|2|0|2|0|—|—
[ROW] 10: 1914|Southern Intercollegiate|George C. Rogers|2|5|0|0|3|0|—|—
[ROW] 11: 1915|Southern Intercollegiate|George C. Rogers|5|3|0|2|3|0|—|—
[ROW] 12: 1916|Southern Intercollegiate|Harvey O'Brien|6|1|1|4|1|0|—|—
[ROW] 13: 1917|Southern Intercollegiate|Harvey O'Brien|3|3|0|1|3|0|—|—
[ROW] 14: 1918|Southern Intercollegiate|Harvey O'Brien|0|2|1|0|1|1|—|—
[ROW] 15: 1919|Southern Intercollegiate|George C. Rogers|4|4|1|2|4|1|—|—
[ROW] 16: 1920|Southern Intercollegiate|Harvey O'Brien|2|6|0|1|5|0|—|—
[ROW] 17: 1921|Southern Intercollegiate|Harvey O'Brien|3|3|2|2|3|1|—|—
[ROW] 18: 1922|Southern Intercollegiate|Carl Prause|3|5|0|1|2|0|—|—
...
[ROW] 110: 2014|Southern|Mike Houston|Upcoming|Upcoming|Upcoming|Upcoming|Upcoming|Upcoming|Upcoming|Upcoming
[ROW] 111: Totals: 105 Seasons|2 Conferences|23 Head Coaches|Total Wins 473|Total Losses 536|Total Ties 32|239 Conference Wins 55 SIAA 184 SoCon|379 Conference Losses 58 SIAA 321 SoCon|13 Conference Ties 8 SIAA 5 SoCon|Regular Season Champions 2 times|1–0 Bowl Record 1–3 Playoff Record

Answer the following question based on the data above: "which year did the team have their most total wins?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

SQL: ```SELECT season, total_wins FROM DF;```.

The database table DF is shown as follows:
[HEAD]: season|total_wins
---
[ROW] 1: 1905|2
[ROW] 2: 1906|3
[ROW] 3: 1907|1
[ROW] 4: 1908|4
[ROW] 5: 1909|4
[ROW] 6: 1910|3
[ROW] 7: 1911|5
[ROW] 8: 1912|3
[ROW] 9: 1913|3
[ROW] 10: 1914|2
[ROW] 11: 1915|5
[ROW] 12: 1916|6
[ROW] 13: 1917|3
[ROW] 14: 1918|0
[ROW] 15: 1919|4
[ROW] 16: 1920|2
[ROW] 17: 1921|3
[ROW] 18: 1922|3
...
[ROW] 110: 2014|Upcoming
[ROW] 111: Totals: 105 Seasons|Total Wins 473

Answer the following question based on the data above: "which year did the team have their most total wins?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or directly answering the question.

"""

import dotenv
config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY']
original_output = openai.Completion.create(engine='davinci-codex-002-msft',
                                            prompt=prompt,
                                            max_tokens=1024,
                                            temperature=0,
                                            top_p=1,
                                            frequency_penalty=0.3,
                                            n=1,
                                            stream=False,
                                            stop='```.')

APIConnectionError: Error communicating with OpenAI

In [ ]:
original_output